In [ ]:
# Colab-Specific Setup: Mount Drive and Install Dependencies
from google.colab import drive
drive.mount('/content/drive')

# Install missing packages (Colab has TF/Keras pre-installed)
!pip install -q tensorflow-datasets seaborn scikit-learn

print("✓ Drive mounted and dependencies installed.")

In [ ]:
# Environment Setup and Imports
import os
import time
import random
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow_datasets as tfds

print("TensorFlow:", tf.__version__)
print("TFDS:", tfds.__version__)

# GPU configuration: enable memory growth if GPU available
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✓ Enabled memory growth on {len(physical_gpus)} GPU(s)")
    except Exception as e:
        print("GPU config error:", e)
else:
    print("No GPU detected; running on CPU.")

# Set global seeds for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Mixed precision for faster training on compatible GPUs
if physical_gpus:
    try:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("✓ Mixed precision enabled.")
    except Exception:
        print("Mixed precision not enabled.")

# Set artifact directory to Google Drive (persistent across sessions)
ARTIFACTS_BASE = pathlib.Path('/content/drive/MyDrive/tf_project_artifacts')
ARTIFACTS_BASE.mkdir(parents=True, exist_ok=True)
print(f"✓ Artifacts will be saved to: {ARTIFACTS_BASE}")

In [ ]:
# Load and Inspect Dataset (TFDS - CIFAR-10)
(ds_train_full, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Create validation split from train
VAL_FRACTION = 0.1
train_size = int((1 - VAL_FRACTION) * ds_info.splits['train'].num_examples)

ds_train = ds_train_full.take(train_size)
ds_val = ds_train_full.skip(train_size)

class_names = ds_info.features['label'].names
print("Classes:", class_names)
print("Train size:", train_size)
print("Val size:", ds_info.splits['train'].num_examples - train_size)
print("Test size:", ds_info.splits['test'].num_examples)

# Show a small grid of sample images
plt.figure(figsize=(6,6))
for i, (image, label) in enumerate(ds_train.take(9)):
    plt.subplot(3,3,i+1)
    plt.imshow(image)
    plt.title(class_names[label])
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Image Preprocessing Pipeline (Resize, Normalize, Augment)
IMG_SIZE = 224
NUM_CLASSES = 10

# Basic preprocess: resize to 224x224 and normalize to [0,1]
def preprocess_basic(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0  # normalize
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

# Augmentations: random flip, rotation, zoom, contrast
# Justification: improves generalization, combats overfitting, and simulates viewpoint variations
augment_layers = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

def preprocess_with_augment(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    image = augment_layers(image)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

In [ ]:
# Build Baseline CNN Model (from scratch)
def build_baseline_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(32, 3, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Conv2D(64, 3, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(64, 3, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.25),

        tf.keras.layers.Conv2D(128, 3, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Dropout(0.3),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU(),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(num_classes, dtype='float32'),  # logits
    ])
    return model

baseline_model = build_baseline_cnn()
baseline_model.build((None, IMG_SIZE, IMG_SIZE, 3))
baseline_model.summary()

In [ ]:
# Build Transfer Learning Model (MobileNetV2)
def build_mobilenetv2_head(input_shape=(IMG_SIZE, IMG_SIZE, 3), num_classes=NUM_CLASSES, fine_tune_at=None):
    base = tf.keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
    x = base(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation=None, dtype='float32')(x)
    model = tf.keras.Model(inputs, outputs)

    # Optional fine-tuning of last N layers
    if fine_tune_at is not None and isinstance(fine_tune_at, int):
        base.trainable = True
        for layer in base.layers[:-fine_tune_at]:
            layer.trainable = False
    return model

transfer_model = build_mobilenetv2_head()
transfer_model.summary()

In [ ]:
# Compile Models (optimizer, loss, metrics)
from tensorflow.keras import optimizers

LR = 1e-3

baseline_model.compile(
    optimizer=optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
transfer_model.compile(
    optimizer=optimizers.Adam(learning_rate=LR),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
print("Models compiled.")

In [ ]:
# Prepare tf.data Pipelines and Batching
BATCH_SIZE = 64
AUTO = tf.data.AUTOTUNE

train_ds = ds_train.map(preprocess_with_augment, num_parallel_calls=AUTO)
train_ds = train_ds.cache().shuffle(10_000, seed=SEED).batch(BATCH_SIZE).prefetch(AUTO)

val_ds = ds_val.map(preprocess_basic, num_parallel_calls=AUTO)
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(AUTO)

test_ds = ds_test.map(preprocess_basic, num_parallel_calls=AUTO)
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTO)

print(train_ds, val_ds, test_ds)

In [ ]:
# Train Models with Callbacks (early stopping, LR scheduling)
ckpt_dir = ARTIFACTS_BASE / 'checkpoints'
ckpt_dir.mkdir(parents=True, exist_ok=True)

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath=str(ckpt_dir / 'baseline.keras'), save_best_only=True, monitor='val_accuracy'),
]

start_time = time.time()
hist_baseline = baseline_model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=callbacks)
baseline_time = time.time() - start_time

callbacks_transfer = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3),
    tf.keras.callbacks.ModelCheckpoint(filepath=str(ckpt_dir / 'transfer.keras'), save_best_only=True, monitor='val_accuracy'),
]

start_time = time.time()
hist_transfer = transfer_model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=callbacks_transfer)
transfer_time = time.time() - start_time

print(f"Baseline total training time: {baseline_time:.1f}s")
print(f"Transfer total training time: {transfer_time:.1f}s")

In [ ]:
# Plot Training and Validation Curves
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Accuracy
axs[0].plot(hist_baseline.history['accuracy'], label='Baseline train')
axs[0].plot(hist_baseline.history['val_accuracy'], label='Baseline val')
axs[0].plot(hist_transfer.history['accuracy'], label='Transfer train')
axs[0].plot(hist_transfer.history['val_accuracy'], label='Transfer val')
axs[0].set_title('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].legend()

# Loss
axs[1].plot(hist_baseline.history['loss'], label='Baseline train')
axs[1].plot(hist_baseline.history['val_loss'], label='Baseline val')
axs[1].plot(hist_transfer.history['loss'], label='Transfer train')
axs[1].plot(hist_transfer.history['val_loss'], label='Transfer val')
axs[1].set_title('Loss')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Loss')
axs[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate Models on Test Set
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# Evaluate
test_loss_base, test_acc_base = baseline_model.evaluate(test_ds, verbose=0)
test_loss_trans, test_acc_trans = transfer_model.evaluate(test_ds, verbose=0)
print(f"Baseline - Test Acc: {test_acc_base:.4f}, Loss: {test_loss_base:.4f}")
print(f"Transfer - Test Acc: {test_acc_trans:.4f}, Loss: {test_loss_trans:.4f}")

# Classification report and confusion matrix
# Collect predictions and true labels
true_labels = []
pred_base = []
pred_trans = []
for images, labels in test_ds:
    y_true = tf.argmax(labels, axis=1).numpy()
    true_labels.extend(y_true)
    logits_b = baseline_model.predict(images, verbose=0)
    logits_t = transfer_model.predict(images, verbose=0)
    pred_base.extend(np.argmax(logits_b, axis=1))
    pred_trans.extend(np.argmax(logits_t, axis=1))

print("Baseline classification report:\n", classification_report(true_labels, pred_base, target_names=class_names))
print("Transfer classification report:\n", classification_report(true_labels, pred_trans, target_names=class_names))

cm_base = confusion_matrix(true_labels, pred_base)
cm_trans = confusion_matrix(true_labels, pred_trans)

fig, axes = plt.subplots(1,2, figsize=(14,5))
sns.heatmap(cm_base, ax=axes[0], cmap='Blues', annot=False)
axes[0].set_title('Baseline Confusion Matrix')
sns.heatmap(cm_trans, ax=axes[1], cmap='Greens', annot=False)
axes[1].set_title('Transfer Confusion Matrix')
plt.show()

In [ ]:
# Speed Benchmarking (training time per epoch, throughput)
import pandas as pd

# Approximate per-epoch time using history logs length
epochs_base = len(hist_baseline.history['accuracy'])
epochs_trans = len(hist_transfer.history['accuracy'])

images_per_epoch = train_size  # approximate number of training samples per epoch
throughput_base = images_per_epoch / (baseline_time / max(epochs_base, 1))
throughput_trans = images_per_epoch / (transfer_time / max(epochs_trans, 1))

summary = pd.DataFrame({
    'Model': ['Baseline', 'Transfer'],
    'Epochs': [epochs_base, epochs_trans],
    'TotalTrainTime(s)': [baseline_time, transfer_time],
    'ImagesPerEpoch': [images_per_epoch, images_per_epoch],
    'ApproxThroughput(img/s)': [throughput_base, throughput_trans],
    'TestAcc': [test_acc_base, test_acc_trans],
})
print(summary)

In [ ]:
# Model Comparison (accuracy, speed, robustness)
val_acc_base = np.array(hist_baseline.history['val_accuracy'])
val_acc_trans = np.array(hist_transfer.history['val_accuracy'])

comparison = pd.DataFrame({
    'Metric': ['TestAcc', 'ValAccStd', 'ApproxThroughput'],
    'Baseline': [test_acc_base, float(val_acc_base.std()), float(throughput_base)],
    'Transfer': [test_acc_trans, float(val_acc_trans.std()), float(throughput_trans)],
})
print(comparison)

# Simple bar plot for test accuracy
plt.figure(figsize=(5,4))
plt.bar(['Baseline', 'Transfer'], [test_acc_base, test_acc_trans], color=['steelblue', 'seagreen'])
plt.title('Test Accuracy Comparison')
plt.ylabel('Accuracy')
plt.show()

# Notes: Transfer learning expected to converge faster and achieve higher accuracy
# due to pretrained features; baseline may require more epochs and regularization.

In [ ]:
# Save Artifacts (models, histories, plots) to Google Drive
import json

# Save models
baseline_model.save(ARTIFACTS_BASE / 'baseline_savedmodel')
transfer_model.save(ARTIFACTS_BASE / 'transfer_savedmodel')

# Save histories
with open(ARTIFACTS_BASE / 'hist_baseline.json', 'w') as f:
    json.dump(hist_baseline.history, f)
with open(ARTIFACTS_BASE / 'hist_transfer.json', 'w') as f:
    json.dump(hist_transfer.history, f)

# Save a figure example
fig_path = ARTIFACTS_BASE / 'training_curves.png'
fig = plt.figure(figsize=(8,4))
plt.plot(hist_baseline.history['val_accuracy'], label='Baseline val_acc')
plt.plot(hist_transfer.history['val_accuracy'], label='Transfer val_acc')
plt.legend(); plt.title('Validation Accuracy'); plt.xlabel('Epoch'); plt.ylabel('Acc')
fig.savefig(fig_path)
plt.close(fig)

# Export minimal requirements
reqs_path = ARTIFACTS_BASE / 'requirements.txt'
with open(reqs_path, 'w') as f:
    f.write('tensorflow\n')
    f.write('tensorflow-datasets\n')
    f.write('matplotlib\n')
    f.write('seaborn\n')
    f.write('numpy\n')
    f.write('pandas\n')
    f.write('scikit-learn\n')

print("✓ Artifacts saved to:", ARTIFACTS_BASE)

# Conclusion & Future Work
- Summarize key findings: accuracy, speed, and robustness differences between baseline CNN and MobileNetV2 transfer learning on CIFAR-10.
- Limitations: dataset size, compute constraints, potential overfitting without careful regularization.
- Future work: try fine-tuning deeper layers, experiment with other pretrained models (ResNet50, EfficientNet), add advanced augmentations or regularization, explore Transformers for vision tasks (ViT).

# References
- TensorFlow and Keras documentation
- TensorFlow Datasets (TFDS) CIFAR-10
- MobileNetV2 paper and Keras applications
- General best practices for image preprocessing and transfer learning

# Optional: GAN for CIFAR-10 Truck Class
This section implements a DCGAN focused on generating images of the CIFAR-10 `truck` class. It demonstrates:
- Class-specific data filtering
- Generator (ConvTranspose) and Discriminator (Conv) architectures
- Adversarial training loop with non-saturating loss
- Periodic sample generation and checkpointing

Note: Training a GAN is compute-intensive; for demonstration we use fewer epochs and a subset of data. Increase epochs and dataset size for higher fidelity.

In [ ]:
# GAN: Dataset Filtering for Truck Class
TRUCK_LABEL = class_names.index('truck') if 'truck' in class_names else 9  # fallback
SUBSET_SIZE = 10000  # reduce for quicker demo; set None to use all

# Filter train set for trucks only
truck_train = ds_train_full.filter(lambda img, lbl: tf.equal(lbl, TRUCK_LABEL))
if SUBSET_SIZE:
    truck_train = truck_train.take(SUBSET_SIZE)

# Basic preprocessing (no augmentation for GAN real images)
def gan_preprocess(image, label):
    image = tf.image.resize(image, (32, 32))  # keep native size
    image = tf.cast(image, tf.float32) / 127.5 - 1.0  # scale to [-1,1]
    return image

BATCH_GAN = 128
AUTO = tf.data.AUTOTUNE

gan_ds = truck_train.map(lambda img, lbl: gan_preprocess(img, lbl), num_parallel_calls=AUTO)
gan_ds = gan_ds.shuffle(5000, seed=SEED).batch(BATCH_GAN, drop_remainder=True).prefetch(AUTO)
print(gan_ds)

In [ ]:
# GAN: Generator and Discriminator
LATENT_DIM = 128

# Generator: input z -> 32x32x3 using ConvTranspose
def build_generator(latent_dim=LATENT_DIM):
    inputs = tf.keras.Input(shape=(latent_dim,))
    x = tf.keras.layers.Dense(4*4*512, use_bias=False)(inputs)
    x = tf.keras.layers.Reshape((4, 4, 512))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    outputs = tf.keras.layers.Conv2DTranspose(3, 3, activation='tanh', padding='same')(x)
    return tf.keras.Model(inputs, outputs, name='generator')

# Discriminator: input image -> probability real
def build_discriminator():
    inputs = tf.keras.Input(shape=(32,32,3))
    x = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same')(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1)(x)  # logits
    return tf.keras.Model(inputs, x, name='discriminator')

generator = build_generator()
discriminator = build_discriminator()

generator.summary()
discriminator.summary()

In [ ]:
# GAN: Losses, Optimizers, and Training Step
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

G_LR = 2e-4
D_LR = 2e-4
beta1 = 0.5

g_optimizer = tf.keras.optimizers.Adam(G_LR, beta_1=beta1)
d_optimizer = tf.keras.optimizers.Adam(D_LR, beta_1=beta1)

@tf.function
def gan_train_step(real_images):
    batch_size = tf.shape(real_images)[0]
    random_latent = tf.random.normal((batch_size, LATENT_DIM))

    with tf.GradientTape(persistent=True) as tape:
        fake_images = generator(random_latent, training=True)
        real_logits = discriminator(real_images, training=True)
        fake_logits = discriminator(fake_images, training=True)

        d_loss_real = cross_entropy(tf.ones_like(real_logits), real_logits)
        d_loss_fake = cross_entropy(tf.zeros_like(fake_logits), fake_logits)
        d_loss = d_loss_real + d_loss_fake

        g_loss = cross_entropy(tf.ones_like(fake_logits), fake_logits)

    d_grads = tape.gradient(d_loss, discriminator.trainable_variables)
    g_grads = tape.gradient(g_loss, generator.trainable_variables)

    d_optimizer.apply_gradients(zip(d_grads, discriminator.trainable_variables))
    g_optimizer.apply_gradients(zip(g_grads, generator.trainable_variables))

    return d_loss, g_loss

# Fixed noise for monitoring progress
FIXED_LATENT = tf.random.normal((16, LATENT_DIM))

In [ ]:
# GAN: Training Loop and Sample Generation
EPOCHS_GAN = 10  # Increase for better quality (try 50+ on Colab GPU)
SAMPLES_DIR = ARTIFACTS_BASE / 'gan_samples'
SAMPLES_DIR.mkdir(parents=True, exist_ok=True)

for epoch in range(1, EPOCHS_GAN + 1):
    start = time.time()
    d_losses = []
    g_losses = []
    for real_batch in gan_ds:
        d_loss, g_loss = gan_train_step(real_batch)
        d_losses.append(d_loss)
        g_losses.append(g_loss)
    epoch_d = tf.reduce_mean(d_losses)
    epoch_g = tf.reduce_mean(g_losses)

    # Generate monitoring samples
    generated = generator(FIXED_LATENT, training=False)
    generated = (generated + 1.0) / 2.0  # back to [0,1]

    fig, axes = plt.subplots(4,4, figsize=(4,4))
    for i, ax in enumerate(axes.flat):
        ax.imshow(generated[i].numpy())
        ax.axis('off')
    plt.suptitle(f'Epoch {epoch} Samples')
    fig.savefig(SAMPLES_DIR / f'epoch_{epoch:03d}.png')
    plt.close(fig)

    print(f'Epoch {epoch}/{EPOCHS_GAN} D_loss={epoch_d:.4f} G_loss={epoch_g:.4f} time={(time.time()-start):.1f}s')

# Save final generator model
generator.save(ARTIFACTS_BASE / 'gan_generator_savedmodel')
print('✓ GAN training complete. Generator saved to Drive.')